**Disclaimer**
This Jupyter notebook is derived from and builds upon the following notebook. Credit and appreciation are extended to the original author(s) for their foundational work, which has been adapted and expanded for the current purpose.
https://github.com/build-on-aws/langchain-embeddings/blob/main/notebooks/03_build_pgvector_db.ipynb

# Supercharging Vector Similarity Search with Amazon Aurora and pgvector
In this Jupyter Notebook, you'll explore how to store vector embeddings in a vector database using [Amazon Aurora](https://aws.amazon.com/es/rds/aurora/) and the pgvector extension. This approach is particularly useful for applications that require efficient similarity searches on high-dimensional data, such as natural language processing, image recognition, and recommendation systems.

[Amazon Aurora](https://aws.amazon.com/es/rds/aurora/) is a fully managed relational database service provided by Amazon Web Services (AWS). It is compatible with PostgreSQL and supports the [pgvector](https://github.com/pgvector/pgvector) extension, which introduces a 'vector' data type and specialized query operators for vector similarity searches. The pgvector extension utilizes the ivfflat indexing mechanism to expedite these searches, allowing you to store and index up to 16,000 dimensions, while optimizing search performance for up to 2,000 dimensions.

For developers and data engineers with experience in relational databases and PostgreSQL, Amazon Aurora with pgvector offers a powerful and familiar solution for managing vector datastores, especially when dealing with structured datasets. Alternatively, Amazon Relational Database Service (RDS) for PostgreSQL is also a suitable option, particularly if you require specific PostgreSQL versions.

Both Amazon Aurora and Amazon RDS for PostgreSQL offer horizontal scaling capabilities for read queries, with a maximum of 15 replicas. Additionally, Amazon Aurora PostgreSQL provides a Serverless v2 option, which automatically scales compute and memory resources based on your application's demand, simplifying operations and capacity planning.

In [2]:
!pip install -q psycopg[binary] langchain_postgres langchain_community langchain_aws langchain_experimental datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
astropy 6.1.0 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
scikit-image 0.23.2 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
sphinx 7.3.7 requires docutils<0.22,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [10]:
import json
import boto3
import pandas as pd
from datasets import load_dataset

from langchain_community.document_loaders import DataFrameLoader
from langchain.docstore.document import Document
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_postgres import PGVector
from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

**1- Set up database connection:** Ensure that you have an Amazon Aurora instance configured and running. 

In [11]:
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
)

response = client.get_secret_value(SecretId="phoenix-demo-db-credential")
secret = json.loads(response['SecretString'])
print(secret)

{'dbClusterIdentifier': 'phoenix-demo-db', 'password': 'Y535lJ462DlzvIu7SNtVTuWFbjXDcDs9', 'dbname': 'postgres', 'engine': 'postgres', 'port': 5432, 'host': 'phoenix-demo-db.cluster-c56c0icccghq.us-east-1.rds.amazonaws.com', 'username': 'postgres'}


In [12]:
import psycopg

connection = f"postgresql://{secret['username']}:{secret['password']}@{secret['host']}:{secret['port']}/{secret['dbname']}"

# Establish the connection to the database
conn = psycopg.connect(
    conninfo = connection
)
# Create a cursor to run queries
cur = conn.cursor()

**2- Enable the [pgvector extension](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraPostgreSQLReleaseNotes/AuroraPostgreSQL.Extensions.html?sc_channel=el&sc_campaign=genai&sc_geo=mult&sc_country=mult&sc_outcome=acq&sc_content=vector-embeddings-and-rag-demystified-2):** Once connected to your Aurora instance, enable the pgvector extension by running the following SQL command:

In [13]:
cur.execute("CREATE EXTENSION vector;")
conn.commit()

**3- Create a table to store embeddings:** Define a table schema to store your vector embeddings and any associated metadata. 

This table includes columns for a unique identifier (id), the original text (text), and the vector embedding (embedding) with a dimensionality of 1536.

In [14]:
table_name = "embeddings"
query = f"""CREATE TABLE {table_name} (
    id SERIAL PRIMARY KEY,
    text TEXT,
    embedding VECTOR(1536)
);"""
cur.execute(query)
conn.commit()

## Load HuggingFace Dataset to PG Vector Store

In [15]:
bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1") 
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", client=bedrock_client)
llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", client=bedrock_client)

In [16]:
# Load aws_whitepapers dataset from huggingface
ds = load_dataset("si3mshady/aws_whitepapers")

# Convert dataset to dataframe
df = pd.DataFrame(ds["train"])

# Check data shape
df.head(5)

Generating train split:   0%|          | 0/209 [00:00<?, ? examples/s]

,Title,Content,Category
0,Amazon_Aurora_Migration_Handbook,This paper has been archived For the latest Am...,General
1,A_Practical_Guide_to_Cloud_Migration_Migrating...,Archived A Practical Gui de to Cl oud Migratio...,General
2,Amazon_Aurora_MySQL_Database_Administrators_Ha...,This version has been archived For the latest ...,General
3,A_Platform_for_Computing_at_the_Mobile_Edge_Jo...,ArchivedA Platform for Computing at the Mobile...,General
4,Amazon_Elastic_File_System_Choosing_Between_Di...,This paper has been archived For the latest te...,General


In [17]:
def load_and_split_semantic(embeddings):
    loader = DataFrameLoader(df, page_content_column="Content")
    docs = loader.load_and_split()
    print(f"docs:{len(docs)}")
    return docs

In [18]:
# function to create vector store
def create_vectorstore(embeddings, collection_name, conn):
    vectorstore = PGVector(
        embeddings=embeddings,
        collection_name=collection_name,
        connection=conn,
        use_jsonb=True,
    )
    return vectorstore

In [19]:
docs = load_and_split_semantic(bedrock_embeddings)

docs:2918


In [21]:
collection_name_text = "aws_whitepapers"
vectorstore = create_vectorstore(bedrock_embeddings, collection_name_text, connection)

In [ ]:
# Add documents to the vectorstore
# this will take roughly 10-15 minutes.
vectorstore.add_documents(docs)

## Verify successful loading of dataset

In [23]:
vectorstore.similarity_search("what are the pillars in AWS well architected framework?", k=5)

[Document(metadata={'Title': 'AWS_WellArchitected_Framework', 'Category': 'General'}, page_content='ArchivedAWS WellArchitected Framework July 2020 This whitepaper describes the AWS WellArchitected Framework It provides guidance to help cus tomers apply best practices in the design delivery and maintenance of AWS environments We address general design principles as well as specific best practices and guidance in ﬁve conceptual areas that we define as the pillars of the WellArchitected FrameworkThis paper has been archived The latest version is available at: https://docsawsamazoncom/wellarchitected/latest/framework/welcomehtmlArchivedAWS WellArchitected Framework Notices Customers are responsible for making their own independent assessment of the in formation in this document This document: (a) is for informational purposes only (b) represents current AWS product offerings and practices which are subject to change without notice and (c) does not create any commitments or assurances from

In [24]:
vectorstore.similarity_search_with_relevance_scores("what is the durability of s3?", k=5)

[(Document(metadata={'Title': 'AWS_Storage_Services_Overview', 'Category': 'General'}, page_content='to serve as the primary data storage for ArchivedAmazon Web Services – AWS Storage Services Overview Page 5 mission critical data In fact Amazon S3 is designed for 99999999999 percent (11 nines) durability per o bject and 9999 percent availability over a one year period Additionally you have a choice of enabling cross region replication on each Amazon S3 bucket Once enabled cross region replication automatically copies objects across buckets in different AWS Regions asynchronously providing 11 nines of durability and 4 nines of availability on both the source and destination Amazon S3 objects Scalability and Elasticity Amazon S3 has been designed to offer a very high level of automatic scalability and elasti city Unlike a typical file system that encounters issues when storing a large number of files in a directory Amazon S3 supports a virtually unlimited number of files in any bucket A

### Retrieve information using Amazon Bedrock

In [34]:
template = """
You are an AI assistant tasked with answering questions based on provided context. Your goal is to provide accurate and relevant answers using only the information given.

Here is the context you should use to answer the question:

<context>
{context}
</context>

Now, here is the question you need to answer:

<question>
{query}
</question>

Instructions:
1. Carefully read and analyze the provided context.
2. Identify key information in the context that is relevant to the question.
3. Formulate an answer to the question using only the information from the given context.
4. If the context does not contain enough information to fully answer the question, state this clearly in your response.
5. Do not use any external knowledge or information not present in the provided context.
6. Keep your answer concise and to the point, while ensuring it fully addresses the question.

Format your response as follows:
1. Begin with a brief answer to the question.
2. Follow with a more detailed explanation, if necessary.
3. If you're quoting directly from the context, use quotation marks and indicate the quote's location in the context.

Remember, it's important to rely solely on the given context and not to introduce any external information or assumptions in your answer.
"""

In [35]:
query = "what is the durability of s3?"
prompt = ChatPromptTemplate.from_template(template)


def parse_docs(docs):
    return {
        'query': query,
        'context': docs
    }


llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name="us-east-1",
)

chain = vectorstore.as_retriever() | parse_docs | prompt | llm

print(chain.invoke(query))


content='According to the context provided, Amazon S3 is designed for "99.999999999% (11 nines) durability per object and 99.99% availability over a one year period."\n\nThis is stated directly in the following quote from the first document:\n"Amazon S3 is designed for 99.999999999% (11 nines) durability per object and 99.99% availability over a one year period."\n\nThe context also provides some additional details on how Amazon S3 achieves this high durability:\n"Objects are redundantly stored on multiple devices across multiple facilities in an Amazon S3 Region. Once stored, Amazon S3 helps maintain the durability of the objects by quickly detecting and repairing any lost redundancy. Amazon S3 also regularly verifies the integrity of data stored using checksums. If corruption is detected, it is repaired using redundant data."\n\nSo in summary, the context clearly states that Amazon S3 has an extremely high durability of 11 nines (99.999999999%) per object through redundant storage ac

Learn more: 
- [Leverage pgvector and Amazon Aurora PostgreSQL for Natural Language Processing, Chatbots and Sentiment Analysis](https://aws.amazon.com/es/blogs/database/leverage-pgvector-and-amazon-aurora-postgresql-for-natural-language-processing-chatbots-and-sentiment-analysis/)

## Delete vectorDB

In [ ]:
vectorstore.drop_tables()